In [72]:
import pandas as pd

file_path = 'mse_434_paper_data.xlsx'

try:
    df = pd.read_excel(file_path, sheet_name='customers_nonstress')
    print("Data loaded successfully:")
    display(df)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Data loaded successfully:


,Plant,Longitude,Latitude,Daily consumption [T],Safety stock [T]
0,Central,17.00,52.00,—,—
1,2,17.00,54.44,6,6
2,3,15.06,53.50,6,6
3,4,20.52,50.34,12,12
4,5,21.77,50.24,19,19
5,8,22.22,50.92,25,25
6,10,22.82,52.30,20,20
7,12,15.49,53.36,17,17
8,13,22.38,51.93,12,12
9,20,20.01,53.27,7,7


In [73]:
'''
Calculate distance using the Haversine Formula for spherical data / longitude and latitude coordinates
'''

# https://community.esri.com/t5/coordinate-reference-systems-blog/distance-on-a-sphere-the-haversine-formula/ba-p/902128

def haversine(coord1: object, coord2: object):
    import math

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    km = round(km, 3)
    
    return km


In [74]:
n = len(df)
# Convert Plant column to string to ensure consistent types
plant_names = df['Plant'].astype(str).tolist()
distance_matrix = pd.DataFrame(index=plant_names, columns=plant_names)
distance_matrix.index.name = None

# Create distance matrix using haversine() function to show distances between plants and customers (regional warehouses)
for i in range(n):
    for j in range(n):
        coord1 = (df.loc[i, 'Longitude'], df.loc[i, 'Latitude'])
        coord2 = (df.loc[j, 'Longitude'], df.loc[j, 'Latitude'])

        distance_matrix.iloc[i, j] = haversine(coord1, coord2)

# Convert all values to float
distance_matrix = distance_matrix.astype(float)

# Display result
print("Distance Matrix (in kilometers):")
display(distance_matrix)

Distance Matrix (in kilometers):


,Central,2,3,4,5,8,10,12,13,20,28
Central,0.000,271.316,211.806,307.026,386.073,380.964,398.382,182.285,368.592,247.357,423.874
2,271.316,0.000,164.384,514.532,568.155,526.059,453.343,155.584,454.106,236.396,602.020
3,211.806,164.384,0.000,513.228,585.791,565.574,537.020,32.464,522.776,329.214,622.970
4,307.026,514.532,513.228,0.000,89.496,136.146,270.240,481.588,219.303,327.680,125.345
5,386.073,568.155,585.791,89.496,0.000,82.017,240.420,553.586,192.687,358.010,37.833
8,380.964,526.059,565.574,136.146,82.017,0.000,158.942,533.115,112.853,301.751,94.518
10,398.382,453.343,537.020,270.240,240.420,158.942,0.000,506.103,50.944,217.568,251.349
12,182.285,155.584,32.464,481.588,553.586,533.115,506.103,0.000,491.058,300.378,590.718
13,368.592,454.106,522.776,219.303,192.687,112.853,50.944,491.058,0.000,218.660,207.089
20,247.357,236.396,329.214,327.680,358.010,301.751,217.568,300.378,218.660,0.000,387.313


In [75]:
def calculate_route_cost(route_customers, distance_matrix, c_per_km=2.0):
    """
    Calculates the total transportation cost for a given route.

    Parameters:
    - route_customers: list of customer names (as strings), e.g., ['2', '20']
    - distance_matrix: pandas DataFrame of distances in km
    - c_per_km: cost per km (float)

    Returns:
    - total_cost: float 
    """
    if not route_customers:
        return 0.0

    total_km = 0
    depot = "Central"

    # Distance from Central to first customer
    total_km += distance_matrix.loc[depot, route_customers[0]]

    # Distance between intermediate customers
    for i in range(len(route_customers) - 1):
        total_km += distance_matrix.loc[route_customers[i], route_customers[i + 1]]

    # Distance from last customer back to Central
    total_km += distance_matrix.loc[route_customers[-1], depot]

    return round(c_per_km * total_km, 2)


In [76]:
def NN_Route_heuristic(df, distance_matrix, capacity_limit=25, c_per_km=2.0):
    """
    Constructs routes using a Nearest Neighbor heuristic with capacity constraints.

    Parameters:
    - df: DataFrame with 'Plant', 'Daily consumption [T]'
    - distance_matrix: precomputed DataFrame with distances
    - capacity_limit: max vehicle load per route
    - c_per_km: cost per km

    Returns:
    - route_map: dict of route_id → list of customer nodes
    - cj: dict of route_id → total route cost
    - aij: dict of (customer, route_id) → 1
    """
    depot = "Central"
    customers = df[df["Plant"].astype(str) != depot]
    demand_lookup = df.set_index("Plant")["Daily consumption [T]"].dropna().to_dict()
    unvisited = set(str(p) for p in customers["Plant"])
    
    route_map = {}
    cj = {}
    aij = {}
    route_id_counter = 0

    while unvisited:
        route_customers = []
        current_node = depot
        capacity_used = 0

        while True:
            nearest = None
            min_dist = float("inf")

            for candidate in unvisited:
                demand = demand_lookup[int(candidate)]
                if capacity_used + demand <= capacity_limit:
                    dist = distance_matrix.loc[current_node, candidate]
                    if dist < min_dist:
                        min_dist = dist
                        nearest = candidate

            if nearest is None:
                break

            route_customers.append(nearest)
            unvisited.remove(nearest)
            capacity_used += demand_lookup[int(nearest)]
            current_node = nearest

        # Register route
        route_id = f"R{route_id_counter}"
        route_map[route_id] = route_customers
        cj[route_id] = calculate_route_cost(route_customers, distance_matrix, c_per_km)
        for cust in route_customers:
            aij[(cust, route_id)] = 1

        route_id_counter += 1
    
    # Ensure all customers are represented in aij
    all_customers = sorted(set(str(p) for p in df["Plant"] if str(p).lower() != "central"))
    all_routes = list(route_map.keys())

    for i in all_customers:
        for j in all_routes:
            if (i, j) not in aij:
                aij[(i, j)] = 0

    return route_map, cj, aij


In [77]:
# === Constants ===
T = 5  # number of time periods
L = 25  # vehicle capacity (tons)
M = 10000  # Big M
b = 1.2  # inventory holding cost per ton
c_per_km = 2  # transportation cost per km

# === Load customer data ===
customers_df = pd.read_excel("mse_434_paper_data.xlsx", sheet_name="customers_nonstress")

# === Generate routes using Nearest Neighbor heuristic ===
route_map, cj, aij = NN_Route_heuristic(customers_df, distance_matrix, capacity_limit=L, c_per_km=c_per_km)
route_ids = list(route_map.keys())
customers = sorted(set(i for route in route_map.values() for i in route))  

# === Print outputs for debugging ===
print("Route Map:")
for route_id, custs in route_map.items():
    print(f"{route_id}: {custs}")

print("\nRoute Costs (cj):")
print(cj)

print("\naij Mapping:")
print(aij)

# === Build di, gi, bi ===
di = {i: customers_df.loc[customers_df["Plant"] == int(i), "Daily consumption [T]"].values[0] for i in customers}
gi = {i: customers_df.loc[customers_df["Plant"] == int(i), "Safety stock [T]"].values[0] for i in customers}
bi = {i: b for i in customers}


Route Map:
R0: ['12', '3']
R1: ['20', '13', '2']
R2: ['4', '28']
R3: ['8']
R4: ['5']
R5: ['10']

Route Costs (cj):
{'R0': np.float64(853.11), 'R1': np.float64(2382.88), 'R2': np.float64(1712.49), 'R3': np.float64(1523.86), 'R4': np.float64(1544.29), 'R5': np.float64(1593.53)}

aij Mapping:
{('12', 'R0'): 1, ('3', 'R0'): 1, ('20', 'R1'): 1, ('13', 'R1'): 1, ('2', 'R1'): 1, ('4', 'R2'): 1, ('28', 'R2'): 1, ('8', 'R3'): 1, ('5', 'R4'): 1, ('10', 'R5'): 1, ('10', 'R0'): 0, ('10', 'R1'): 0, ('10', 'R2'): 0, ('10', 'R3'): 0, ('10', 'R4'): 0, ('12', 'R1'): 0, ('12', 'R2'): 0, ('12', 'R3'): 0, ('12', 'R4'): 0, ('12', 'R5'): 0, ('13', 'R0'): 0, ('13', 'R2'): 0, ('13', 'R3'): 0, ('13', 'R4'): 0, ('13', 'R5'): 0, ('2', 'R0'): 0, ('2', 'R2'): 0, ('2', 'R3'): 0, ('2', 'R4'): 0, ('2', 'R5'): 0, ('20', 'R0'): 0, ('20', 'R2'): 0, ('20', 'R3'): 0, ('20', 'R4'): 0, ('20', 'R5'): 0, ('28', 'R0'): 0, ('28', 'R1'): 0, ('28', 'R3'): 0, ('28', 'R4'): 0, ('28', 'R5'): 0, ('3', 'R1'): 0, ('3', 'R2'): 0, ('3', 

In [78]:
from gurobipy import Model, GRB

model = Model("JointInventoryTransportation")

# === Indices ===
K = range(1, T + 1)

# === Variables ===

# Route usage decision 
x = model.addVars(route_ids, K, vtype=GRB.BINARY, name="x")                  # x_jk

# Inventory level at customer i in time period k
y = model.addVars(customers, K, lb=0, name="y")                              # y_ik

# Delivered amount at customer i in time period k on route j
z = model.addVars(customers, route_ids, K, lb=0, name="z")                   # z_ijk

# Objective: transport cost + inventory cost
model.setObjective(
    sum(cj[j] * x[j, k] for j in route_ids for k in K) +
    sum(bi[i] * y[i, k] for i in customers for k in K),
    GRB.MINIMIZE
)

In [79]:
# # === Constraint (1): Initial inventory y_i1 = gi_i === (Initial inventory constraint)
# for i in customers:
#     model.addConstr(y[i, 1] == gi[i], name=f"init_inventory_{i}")

# === Constraint (2): z_ijk ≤ aij * x_jk * M === (Deliver only if the route is used)
for i in customers:
    for j in route_ids:
        for k in K:
            model.addConstr(z[i, j, k] <= aij[i, j] * x[j, k] * M, name=f"link_z_x_{i}_{j}_{k}")

# === Constraint (3): sum_i z_ijk ≤ Q for each route j, period k === (Truck capacity constraint)
for j in route_ids:
    for k in K:
        model.addConstr(sum(z[i, j, k] for i in customers) <= L, name=f"truck_cap_{j}_{k}")

# === Constraint (4): Inventory balance for period 1 to T-1 === (Inventory continuity constraint for periods 1 to T-1)
for i in customers:
    for k in range(1, T):
        model.addConstr(
            y[i, k+1] == y[i, k] + sum(z[i, j, k] for j in route_ids) - di[i],
            name=f"inventory_flow_{i}_{k}"
        )

# === Constraint (5): Cyclic inventory from T to 1 === (Cyclic inventory constraint for period T to 1)
for i in customers:
    model.addConstr(
        y[i, 1] == y[i, T] + sum(z[i, j, T] for j in route_ids) - di[i],
        name=f"cyclic_inventory_{i}"
    )

# === Constraint (6): Minimum inventory ≥ gi === (Safety stock constraint)
for i in customers:
    for k in K:
        model.addConstr(y[i, k] >= gi[i], name=f"safety_stock_{i}_{k}")
        
# F = 3  # number of trucks available per day

# # Limit number of routes (fleet size) per time period (EXTENSION)
# for k in K:
#     model.addConstr(sum(x[j, k] for j in route_ids) <= F, name=f"fleet_limit_{k}")

# Extension Idea: Constraint the number of trucks + decision variable for serving through common carrier if distance is too long
# Similar problem 



In [80]:
model.optimize()

# Output
print(f"\nTotal cost: {model.ObjVal:.2f}\n")

print("Route usage:")
for k in K:
    print(f"\nPeriod {k}:")
    for j in route_ids:
        if x[j, k].X > 0.5:
            print(f"  Route {j} used")

print("\nInventory levels:")
for i in customers:
    for k in K:
        print(f"Customer {i}, Period {k} → {y[i, k].X:.2f} units")

print("\nDeliveries z_ijk:")
for i in customers:
    for j in route_ids:
        for k in K:
            if z[i, j, k].X > 0.01:
                print(f"Deliver {z[i, j, k].X:.2f} to Customer {i} via Route {j} in Period {k}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 430 rows, 380 columns and 1100 nonzeros
Model fingerprint: 0xcf5e02f6
Variable types: 350 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+00, 2e+01]
Found heuristic solution: objective 49163.200000
Presolve removed 410 rows and 345 columns
Presolve time: 0.01s
Presolved: 20 rows, 35 columns, 65 nonzeros
Found heuristic solution: objective 44250.490000
Variable types: 30 continuous, 5 integer (5 binary)

Root relaxation: objective 4.414249e+04, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0  